In [ ]:
pip install boto3 pandas openpyxl xlrd gspread oauth2client

In [39]:
import pandas as pd
import boto3
import os
from dotenv import load_dotenv
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from openpyxl import load_workbook

In [40]:
load_dotenv()

True

In [41]:
access_key = os.getenv("AWS_ACCESS_KEY_ID")
secret_key = os.getenv("AWS_SECRET_ACCESS_KEY")
BUCKET_NAME = os.getenv("BUCKET_NAME")

In [42]:
s3 = boto3.client(
    "s3",
    aws_access_key_id = access_key ,
    aws_secret_access_key = secret_key 
)


In [43]:
def download_xl_file(bucket_name , s3_key , local_file) :
    try :
        s3.download_file(bucket_name , s3_key , local_file)
        print ("download success")
    except Exception as e :
        print (f"error : '{e}'")

In [44]:
def upload_csv_file(bucket_name , s3_key , local_file) :
    try :
        s3.upload_file(local_file , bucket_name , s3_key)
        print ("upload success")
    except Exception as e :
        print (f"error : '{e}'")

In [45]:
def empty_xl_file(xl_file) :
    wb = load_workbook(xl_file)
    ws = wb.active
    if ws.max_row > 1 :
        ws.delete_rows(2 , ws.max_row - 1)
    wb.save(xl_file)

In [68]:
s3_key = "sondagepublic/sondage.xlsx"
local_file = 'sondage.xlsx'
download_xl_file(BUCKET_NAME , s3_key , local_file)

download success


In [69]:
data = pd.read_excel(local_file , engine = "openpyxl")
print(data)

                          product  quantity        date  price_before  \
0                QMobile Noir A8i        19  03/04/2025          4.29   
1    Lenovo Golden Warrior Note 8         2  11/12/2024          1.99   
2       Xiaomi Redmi Y1 (Note 5A)        15  24/03/2025          1.99   
3                QMobile Noir M90        17  01/04/2025          0.99   
4                       Celkon A7        10  31/07/2024          4.99   
..                            ...       ...         ...           ...   
995                Micromax GC400        12  27/02/2025          6.49   
996        Motorola Moto Z4 Force         3  02/04/2025         14.99   
997                  Maxon MX-A30         7  23/08/2024         24.99   
998                 Samsung E1125         6  11/03/2025          5.49   
999            Acer Liquid Jade 2        18  14/06/2024          3.99   

     price_after    prom_type gender  age  income  
0          79.99          NaN  Femme   30    6668  
1           2.29   

In [48]:
def delete_empty_row_col(df) :
    df.dropna(how = 'all' , inplace = True)
    df.dropna(axis=1 , how = 'all' , inplace = True)


In [49]:
def rename_col(df) :
    df.columns = df.columns.str.strip().str.lower().str.replace(' ','_')

In [50]:
def delete_duplicate(df) :
    df.drop_duplicates(inplace=True)

In [51]:
def replace_product(df) :
    df["product"] = df["product"].fillna("unknown")


In [52]:
def replace_qty(df) :
    df["quantity"] = df["quantity"].fillna(1) 

In [53]:
def ajust_prices(df) :
    df.dropna(subset=["price_before"] , inplace=True)
    df.dropna(subset=["price_after"] , inplace = True)
    before = df["price_before"]
    after = df["price_after"]
    mask = df["price_after"] > df["price_before"]
    temp = df.loc[mask, "price_before"].copy()
    df.loc[mask, "price_before"] = df.loc[mask, "price_after"]
    df.loc[mask, "price_after"] = temp

In [70]:
data = data[data['prom_type'].notna() & (data['prom_type'] != '')]
delete_empty_row_col(data)
rename_col(data) 
delete_duplicate(data)
replace_product(data)
replace_qty(data)
ajust_prices(data)

In [71]:
data.to_csv('clean_data.csv' , index=False)

In [72]:
upload_csv_file(BUCKET_NAME , "sondagepublic/clean_sondage.csv" , "clean_data.csv")

upload success


In [57]:
print(data)

                    product  quantity        date  price_before  price_after  \
0    Vodafone V860 Smart II         4  09/05/2024          4.99         3.49   
1                  Meizu V8        15  15/05/2024        149.99         5.59   
2         Micromax Canvas 1         4  18/10/2024          6.49         3.49   
3            alcatel OT-310         8  23/05/2024         39.99        24.99   
4               Siemens C35        18  26/02/2025         24.99         2.29   
..                      ...       ...         ...           ...          ...   
995          verykool i121C         5  05/09/2024          7.99         7.49   
996      Micromax Bolt Q339        18  16/01/2025         44.99        19.99   
997            QMobile B800         9  26/01/2025         45.00        39.99   
998              Wiko Robby         9  23/01/2025         99.99        25.99   
999       Kyocera DuraForce        13  01/01/2025         39.99        14.99   

         prom_type gender  income  
0  

In [58]:
print(data)

                    product  quantity        date  price_before  price_after  \
0    Vodafone V860 Smart II         4  09/05/2024          4.99         3.49   
1                  Meizu V8        15  15/05/2024        149.99         5.59   
2         Micromax Canvas 1         4  18/10/2024          6.49         3.49   
3            alcatel OT-310         8  23/05/2024         39.99        24.99   
4               Siemens C35        18  26/02/2025         24.99         2.29   
..                      ...       ...         ...           ...          ...   
995          verykool i121C         5  05/09/2024          7.99         7.49   
996      Micromax Bolt Q339        18  16/01/2025         44.99        19.99   
997            QMobile B800         9  26/01/2025         45.00        39.99   
998              Wiko Robby         9  23/01/2025         99.99        25.99   
999       Kyocera DuraForce        13  01/01/2025         39.99        14.99   

         prom_type gender  income  
0  

In [ ]:
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name('sheet_secret.json', scope)
client = gspread.authorize(creds)
spreadsheet = client.open("analyse") 
worksheet = spreadsheet.sheet1
for row in data.values.tolist():
    worksheet.append_row(row)

print("Les données ont été téléchargées dans Google Sheets.")

In [75]:
path = "sondage.xlsx"
empty_xl_file('sondage.xlsx')
upload_csv_file(BUCKET_NAME , "sondagepublic/sondage.xlsx" , path)

upload success
